In [1]:
def post_money_safe(val, inv):
    return inv / val * 100

def pre_money_safe(pre, stake, inv):
    return stake * pre / (pre+inv)


# B brings in 10MM on a 100MM post money valuation on ABC
# how much B owns?

post_val = 100
new_inv = 10

b_post_stake = post_money_safe(post_val, new_inv)
print(b_post_stake)

# Investor A had 10% stake on ABC (10MM investment with a 100MM pre money Valuation)
# B brings in an additional 10MM on a new round
# how much A still have?

a_pre_stake = 50
pre_val = 90

a_post_stake = pre_money_safe(pre_val, a_pre_stake, new_inv)
print(a_post_stake)

10.0
45.0


In [78]:
class Investment:

    def __init__(self, invested, round):
        self.invested = invested
        self.round = round
        self.shares = self.invested / self.round.price

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, invested, round):
        investment = Investment(invested, round)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments:
            shares += i.shares 
        return shares
    
    def total_invested(self):
        invested = 0
        for i in self.investments:
            invested += i.invested 
        return invested
        
class Round:

    def __init__(self, name, valuation, shares, covenants = None):
        self.name = name
        self.valuation = valuation
        self.shares = shares
        self.price = valuation/shares
        self.covenants = covenants

class Deal:

    def __init__(self, investment, stake):
        self.investment = investment
        self.stake = stake / 100

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name

        self.incorporation = Round('incorporation', equity, shares)

        self.founder = Investor('founder')
        self.founder.new_investment(equity, self.incorporation)

        self.investors = [self.founder]
        self.rounds = [self.incorporation]

    def current_stake(self):
        cap = []
        for i in self.investors:
            last_round = self.rounds[-1]
            total_shares = last_round.shares
            investor_shares = i.total_shares()
            cap.append({
                'investor': i.name,
                'stake': investor_shares / total_shares * 100
            })
        return cap

    def get_investor(self, name):
        for i in self.investors:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.investors.append(new_investor)
        return new_investor

    def get_add_investor(self, name):
        investor = self.get_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def new_round(self, name, investors, deal, covenant=None):

        post_money_valuation = deal.investment / deal.stake
        pre_money_valuation = post_money_valuation -  deal.investment
        
        total_shares =  self.rounds[-1].shares * ( 1 + deal.investment / pre_money_valuation)
        
        round = Round(name, post_money_valuation, total_shares, covenants = covenant)
        
        self.rounds.append(round)

        for i in investors:
            new_investor = self.get_add_investor(i['name'])
            new_investor.new_investment(deal.investment * (i['stake']/100), round)
            
        return None



In [137]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [138]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 100.0}]

In [139]:
deal = Deal(10000, 10)
investors = [{'name': 'ze', 'stake': 100}]
company.new_round('seed', investors, deal)

deal = Deal(500000, 20)
investors = [{'name': 'tera', 'stake': 80}, {'name': 'patria', 'stake': 20}]
company.new_round('a', investors, deal, {'liquidity_preference': 2})

In [140]:
company.current_stake()

[{'investor': 'founder', 'stake': 72.0},
 {'investor': 'ze', 'stake': 8.000000000000002},
 {'investor': 'tera', 'stake': 16.0},
 {'investor': 'patria', 'stake': 4.0}]

In [142]:
exit_multiple = 2
last_valuation = company.rounds[-1].valuation 
total_shares = company.rounds[-1].shares
exit_valuation = last_valuation*exit_multiple
print('total shares: %s' % total_shares)
print('last valuation: %s' % last_valuation)
print('---------------------------------------------')
print('investor', '|', 'round', '|', 'invested', '|', 'stakes', '|', 'covenants')
for i in company.investors:
    for a in i.investments:
        print(i.name, '|', a.round.name, '|', a.invested, '|', a.shares/total_shares*100, '|', a.round.covenants)

print('---------------------------------------------')

total shares: 1388888.888888889
last valuation: 2500000.0
---------------------------------------------
investor | round | invested | stakes | covenants
founder | incorporation | 10000.0 | 72.0 | None
ze | seed | 10000.0 | 8.000000000000002 | None
tera | a | 400000.0 | 16.0 | {'liquidity_preference': 2}
patria | a | 100000.0 | 4.0 | {'liquidity_preference': 2}
---------------------------------------------
